In [1]:
#Open Balanced data that was saved
import pandas as pd

df = pd.read_csv('balanced_data (1).csv')


Some usernames were manually labeled to improve the training of the BERT model. These usernames fall under vet category but are students and technicians thus should be classified under Others.

In [2]:
usernames_to_filter = [
    'Ecstatic-Operation85', 'matcha-fiend', 'Shemoose', 'Active_Pitch4104', 'almostdonestudent',
    'According-Anybody-91', 'DarthChily', 'cappy267', 'Few-Depth-3039', 'Serious_Passage_1260',
    'f4un4', 'mojoburquano', 'Play_Persevere', 'Timsterific24', 'Greyscale_cats', 'glindsaynz',
    'eptesicusfscus', 'Asleep_Leopard182', 'Julitania', 'Gutterrrslut', 'Western_Gift6401'
]

# Filter the DataFrame
filtered_df = df[df['username'].isin(usernames_to_filter)]

In [3]:
#Length of usernames that fall under this category
len(filtered_df)

21

Next, we resample our dataset getting ready for training. Undersampling the 'others' column since it has the most rows.

In [4]:
sampled_others = df[df['standardized_label'] == 'Others'].sample(n=140, random_state=42)
sampled_veterinarian = df[df['standardized_label'] == 'Veterinarian']
sampled_medical_doctor = df[df['standardized_label'] == 'Medical Doctor']

# Combine the samples
sampled_df = pd.concat([filtered_df, sampled_others, sampled_veterinarian, sampled_medical_doctor])

# Reset the index
sampled_df.reset_index(drop=True, inplace=True)
sampled_df.standardized_label.value_counts()

standardized_label
Others            161
Medical Doctor    143
Veterinarian      108
Name: count, dtype: int64

Installing some packages needed.

In [5]:
!pip install accelerate>=0.21.0

In [6]:
!pip install transformers[torch]

In [9]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 9.9 MB/s eta 0:00:00


Beginning the process of model building.


In [16]:
#Import libraries
import optuna
from transformers import TrainingArguments, Trainer
from sklearn.model_selection import KFold
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification
import torch
from torch.utils.data import Dataset
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Sample dataset
data = sampled_df.copy()

# Extract features and labels
X = data['processed_comments']
y = data['standardized_label']

# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split data into train and test data
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.25, random_state=42)

# Initialize the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Define the custom dataset class
class RedditDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Define compute_metrics function
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

def objective(trial):
    # Define hyperparameters to tune
    num_train_epochs = trial.suggest_int('num_train_epochs', 6, 10)
    per_device_train_batch_size = trial.suggest_categorical('per_device_train_batch_size', [8, 16])
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-4, log=True)
    weight_decay = trial.suggest_float('weight_decay', 0.001, 0.1)
    gradient_accumulation_steps = trial.suggest_int('gradient_accumulation_steps', 1, 4)
    warmup_steps = trial.suggest_int('warmup_steps', 0, 1000)
    dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.3)

    # Initialize KFold
    kf = KFold(n_splits=3)
    fold_losses = []

    for train_index, val_index in kf.split(X_train):
        X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]
        y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]

        # Tokenize the data
        train_encodings = tokenizer(list(X_train_fold), truncation=True, padding=True, max_length=512)
        val_encodings = tokenizer(list(X_val_fold), truncation=True, padding=True, max_length=512)

        # Create dataset objects
        train_dataset = RedditDataset(train_encodings, y_train_fold)
        val_dataset = RedditDataset(val_encodings, y_val_fold)

        training_args = TrainingArguments(
            output_dir='./results',
            num_train_epochs=num_train_epochs,
            per_device_train_batch_size=per_device_train_batch_size,
            learning_rate=learning_rate,
            per_device_eval_batch_size=8,
            warmup_steps=warmup_steps,
            weight_decay=weight_decay,
            logging_dir='./logs',
            logging_steps=10,
            evaluation_strategy="epoch",
            save_strategy="epoch",
            gradient_accumulation_steps=gradient_accumulation_steps,
            load_best_model_at_end=True,
            metric_for_best_model="eval_loss",
        )

        model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(label_encoder.classes_))

        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=train_dataset,
            eval_dataset=val_dataset,
            compute_metrics=compute_metrics
        )

        trainer.train()
        eval_result = trainer.evaluate()
        fold_losses.append(eval_result['eval_loss'])

    return np.mean(fold_losses)

In [17]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=8)

print(f'Best trial: {study.best_trial.params}')

[I 2024-05-20 11:39:16,488] A new study created in memory with name: no-name-2e1e0564-7a00-4bf7-9028-060eebfa0d7e


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,1.091200,1.073714,0.427184,0.337393,0.309034,0.427184
2,1.081500,1.065169,0.475728,0.388640,0.351136,0.475728
3,1.092400,1.050402,0.563107,0.472811,0.413299,0.563107
4,1.053800,1.028788,0.640777,0.545270,0.474771,0.640777
5,1.056900,0.994637,0.679612,0.582403,0.513802,0.679612
6,1.005800,0.942674,0.679612,0.582403,0.513802,0.679612
7,0.952900,0.902074,0.689320,0.591547,0.523919,0.689320
8,0.931100,0.874170,0.699029,0.600743,0.534434,0.699029


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classif

...

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
0,No log,1.118668,0.368932,0.198857,0.136111,0.368932
1,No log,1.117079,0.368932,0.198857,0.136111,0.368932
2,No log,1.114220,0.368932,0.198857,0.136111,0.368932
4,1.144500,1.102458,0.368932,0.198857,0.136111,0.368932
5,1.144500,1.094869,0.368932,0.198857,0.136111,0.368932
6,1.121300,1.085716,0.368932,0.198857,0.136111,0.368932
8,1.121300,1.066144,0.378641,0.220466,0.410651,0.378641


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classif

[I 2024-05-20 14:02:06,685] Trial 7 finished with value: 1.0957351525624592 and parameters: {'num_train_epochs': 9, 'per_device_train_batch_size': 16, 'learning_rate': 2.6785581369449074e-05, 'weight_decay': 0.044020545118521284, 'gradient_accumulation_steps': 4, 'warmup_steps': 755, 'dropout_rate': 0.13641904352734474}. Best is trial 6 with value: 0.633239726225535.


Best trial: {'num_train_epochs': 10, 'per_device_train_batch_size': 8, 'learning_rate': 7.891426108585595e-05, 'weight_decay': 0.04491877838405888, 'gradient_accumulation_steps': 4, 'warmup_steps': 733, 'dropout_rate': 0.1411093280797467}


In [25]:
# Extract the best hyperparameters
best_trial = study.best_trial

best_num_train_epochs = 14
best_per_device_train_batch_size = best_trial.params['per_device_train_batch_size']
best_learning_rate = best_trial.params['learning_rate']
best_weight_decay = best_trial.params['weight_decay']
best_gradient_accumulation_steps = best_trial.params['gradient_accumulation_steps']
best_warmup_steps = best_trial.params['warmup_steps']
best_dropout_rate = best_trial.params['dropout_rate']

# Tokenize the full dataset
full_train_encodings = tokenizer(list(X_train), truncation=True, padding=True, max_length=512)
full_train_dataset = RedditDataset(full_train_encodings, y_train)

# Tokenize the test dataset
test_encodings = tokenizer(list(X_test), truncation=True, padding=True, max_length=512)
test_dataset = RedditDataset(test_encodings, y_test)

# Define training arguments using the best hyperparameters
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=best_num_train_epochs,
    per_device_train_batch_size=best_per_device_train_batch_size,
    learning_rate=best_learning_rate,
    per_device_eval_batch_size=8,
    warmup_steps=best_warmup_steps,
    weight_decay=best_weight_decay,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    gradient_accumulation_steps=best_gradient_accumulation_steps,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
)

# Initialize the Trainer with the best hyperparameters
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(label_encoder.classes_))
model.config.hidden_dropout_prob = best_dropout_rate
model.config.attention_probs_dropout_prob = best_dropout_rate

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=full_train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

# Train and evaluate the model
trainer.train()
trainer.evaluate()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
0,No log,1.266932,0.184466,0.057457,0.034028,0.184466
1,1.194600,1.170342,0.233010,0.147116,0.629519,0.233010
2,1.154400,1.031501,0.436893,0.382233,0.676451,0.436893
4,0.976900,0.791360,0.796117,0.806730,0.874771,0.796117
5,0.849700,0.693206,0.834951,0.839761,0.886420,0.834951
6,0.727800,0.589654,0.834951,0.846583,0.905132,0.834951
8,0.523400,0.428866,0.864078,0.869696,0.907294,0.864078
9,0.445500,0.373711,0.864078,0.867974,0.902360,0.864078
10,0.359500,0.345324,0.864078,0.867974,0.902360,0.864078
12,0.269600,0.301510,0.893204,0.897098,0.919592,0.893204


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'eval_loss': 0.3015095293521881,
 'eval_accuracy': 0.8932038834951457,
 'eval_f1': 0.8970979993169215,
 'eval_precision': 0.9195917351257157,
 'eval_recall': 0.8932038834951457,
 'eval_runtime': 3.3302,
 'eval_samples_per_second': 30.929,
 'eval_steps_per_second': 3.904,
 'epoch': 12.923076923076923}

Saving the best model realized

In [26]:
save_directory = './fine-tuned-bert'
trainer.save_model(save_directory)  # Saves the model to the specified directory
tokenizer.save_pretrained(save_directory)  # Saves the tokenizer to the specified directory

print(f'Model and tokenizer saved to {save_directory}')

Model and tokenizer saved to ./fine-tuned-bert


Testing out the model on some sample comments

In [27]:
model_path = './fine-tuned-bert'
model = BertForSequenceClassification.from_pretrained(model_path)
tokenizer = BertTokenizer.from_pretrained(model_path)

# Check if GPU is available and use it; otherwise, use CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Assuming label_encoder is already defined and trained from previous steps
label_encoder = LabelEncoder()
label_encoder.fit(["Others", "Veterinarian", "Medical Doctor"])  # Ensure the order matches your labels

def classify_comment(comment):
    # Preprocess the input comment
    inputs = tokenizer(comment, return_tensors='pt', truncation=True, padding=True, max_length=512)

    # Move inputs to the same device as the model
    inputs = {key: val.to(device) for key, val in inputs.items()}

    # Set the model to evaluation mode
    model.eval()

    # Make predictions
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)

    # Decode the predictions to label names
    predicted_label = label_encoder.inverse_transform(predictions.cpu().numpy())[0]
    return predicted_label

# Example usage
new_comment = "I'm planning to become a vet because I love animals."
classification = classify_comment(new_comment)
print(f'Classification: {classification}')


Classification: Medical Doctor


In [28]:
classification = classify_comment("It's important to stay up-to-date on the latest advances in medical research.")
print(f'Classification: {classification}')

Classification: Medical Doctor


In [29]:
classification = classify_comment("I play bet.")
print(f'Classification: {classification}')

Classification: Others


In [30]:
classification = classify_comment("Hi, LVT here.")
print(f'Classification: {classification}')

Classification: Others
